In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report

import os
import re

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd

import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
#Daten einlesen
RELPATH = '/home/ki-projekt/KI-Projekt/data/TennesseEastman/'

data = []
dataTE = []
index = 0
indexTE = 0

# Dateien einlesen ############# aktuelles Problem: die Daten werden nicht der Reihe nach eingelesen ############
for datei in os.listdir(RELPATH):
    filePath = os.path.join(RELPATH, datei)

    number_match = re.search(r'\d+', datei)
    if number_match:
        number = int(number_match.group())

        #print(f'Found file: {filePath}, Number: {number}')

        with open(filePath, 'r') as file:
            lines = file.read().splitlines()

            if 'te' in datei:
                while len(dataTE) <= number:
                    dataTE.append([])  # Füge neue Listen hinzu, wenn der Index größer als die Länge von dataTE ist
                dataTE[number] = np.transpose([list(map(float, line.split()[:52])) for line in lines])[:, :480]
            else:
                while len(data) <= number:
                    data.append([])  # Füge neue Listen hinzu, wenn der Index größer als die Länge von data ist
                temp_data = [list(map(float, line.split()[:480])) for line in lines]
                if len(temp_data) > 52:
                    data[number] = np.transpose(temp_data)
                else:
                    data[number] = temp_data

matrix_data = np.array(data)
matrix_dataTE = np.array(dataTE)

In [16]:
training_data = []
test_data = []

for i in range(22):
      #label
      oneHotVector = np.eye(22)[i, :]
      fault_data = np.transpose(data[i] / np.max(data))
      fault_dataTE = np.transpose(dataTE[i] / np.max(dataTE))

      for k in range(480):
            training_data.append([fault_data[k], oneHotVector])
            test_data.append([fault_dataTE[k], oneHotVector])

x_train = []
y_train = []

x_test = []
y_test = []

for testvalues, label in training_data:
      x_train.append(testvalues)
      y_train.append(label)

for testvalues, label in test_data:
      x_test.append(testvalues)
      y_test.append(label)

x_train = np.array(x_train)
y_train = np.array(y_train)

x_test = np.array(x_test)
y_test = np.array(y_test)

train_data = TensorDataset(x_train, y_train)
test_data = TensorDataset(x_test, y_test)

TypeError: cannot set 'x_test' attribute of immutable type 'float'

In [13]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(train_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

NameError: name 'dataset' is not defined

In [5]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(52, 52),
            nn.ReLU(),
            nn.Linear(52, 30),
            nn.ReLU(),
            nn.Linear(30, 22)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [7]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

    print("fertig")

In [8]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [9]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------


RuntimeError: mat1 and mat2 must have the same dtype, but got Double and Float